# Healthcare Lab (Pooled Cross Sectional)

**Learning Objectives:**
  * Define and fit simple regression models
  
  * Gain exposure to healthcare related DataSets

## Context of the dataset

### 1. The dataset is consisted of records corresponding to medical events.
### 2. Each medical event is uniquely identified by `MedicalClaim`.
### 3. A given medical event might involve several medical procedures.
### 4. Each medical procedure is uniquely identified by `ClaimItem`
### 5. A given medical procedure is characterized by `PrincipalDiagnosisDesc`,`PrincipalDiagnosis`,`RevenueCodeDesc`, `RevenueCode`, `TypeFlag` and `TotalExpenses`

### 6. Each medical procedure involves: `MemberName`,`MemberID`,`County`,`HospitalName`, `HospitalType`, `StartDate`,`EndDate`


## 1. Library Import

In [1]:
import pandas as pd
import warnings
import numpy as np
import seaborn as sns
import statsmodels.formula.api as smf
from matplotlib import pyplot as plt


In [2]:
warnings.simplefilter('ignore')

## 2. Data loading and DataFrame creation

In [3]:
HealthCareDataSet=pd.read_csv("https://github.com/thousandoaks/Python4DS-I/raw/main/datasets/HealthcareDataset_PublicRelease.csv",sep=',',parse_dates=['StartDate','EndDate','BirthDate'])

In [4]:
HealthCareDataSet.head(3)

,Id,MemberName,MemberID,County,MedicalClaim,ClaimItem,HospitalName,HospitalType,StartDate,EndDate,PrincipalDiagnosisDesc,PrincipalDiagnosis,RevenueCodeDesc,RevenueCode,TypeFlag,BirthDate,TotalExpenses
0,634363,e659f3f4,6a380a28,6f943458,c1e3436737c77899,18,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DET...,636.0,ER,1967-05-13,15.148
1,634364,e659f3f4,6a380a28,6f943458,c1e3436737c77899,21,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DET...,636.0,ER,1967-05-13,3.073
2,634387,e659f3f4,6a380a28,6f943458,c1e3436737c77899,10,04b77561,HOSPITAL,2020-01-08,2020-01-08,Epigastric pain,R10.13,LABORATORY - CLINICAL DIAGNOSTIC: HEMATOLOGY,305.0,ER,1967-05-13,123.900


In [5]:
HealthCareDataSet.describe()

,Id,ClaimItem,StartDate,EndDate,RevenueCode,BirthDate,TotalExpenses
count,52563.000000,52563.000000,52563,52563,52563.000000,52563,52563.000000
mean,685655.197953,12.023990,2020-06-21 00:12:27.902516992,2020-06-24 21:03:58.958963200,386.321995,1948-05-11 14:05:37.378003712,2735.230373
min,634363.000000,1.000000,2020-01-01 00:00:00,2020-01-01 00:00:00,24.000000,1921-01-18 00:00:00,0.000000
25%,658574.500000,5.000000,2020-03-09 00:00:00,2020-03-13 00:00:00,301.000000,1939-11-10 00:00:00,194.642000
50%,684404.000000,10.000000,2020-06-22 00:00:00,2020-06-26 00:00:00,307.000000,1947-05-12 00:00:00,675.262000
75%,712375.500000,16.000000,2020-09-25 00:00:00,2020-09-29 00:00:00,450.000000,1953-12-02 00:00:00,2309.265000
max,741736.000000,127.000000,2020-12-31 00:00:00,2020-12-31 00:00:00,942.000000,1999-08-09 00:00:00,504533.400000
std,30913.839360,10.747558,NaN,NaN,158.551733,NaN,8292.178928


In [6]:
HealthCareDataSet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52563 entries, 0 to 52562
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Id                      52563 non-null  int64         
 1   MemberName              52563 non-null  object        
 2   MemberID                52563 non-null  object        
 3   County                  52563 non-null  object        
 4   MedicalClaim            52563 non-null  object        
 5   ClaimItem               52563 non-null  int64         
 6   HospitalName            52563 non-null  object        
 7   HospitalType            52563 non-null  object        
 8   StartDate               52563 non-null  datetime64[ns]
 9   EndDate                 52563 non-null  datetime64[ns]
 10  PrincipalDiagnosisDesc  52563 non-null  object        
 11  PrincipalDiagnosis      52563 non-null  object        
 12  RevenueCodeDesc         52561 non-null  object

In [7]:
HealthCareDataSet['AgeAtMedicalEvent']=(HealthCareDataSet['StartDate']-HealthCareDataSet['BirthDate'])

In [8]:
HealthCareDataSet['AgeAtMedicalEvent'].dt.total_seconds() / (365.25 * 24 * 60 * 60)

,AgeAtMedicalEvent
0,52.657084
1,52.657084
2,52.657084
3,52.657084
4,52.657084
...,...
52558,80.637919
52559,70.258727
52560,70.258727
52561,70.258727


In [9]:
HealthCareDataSet['AgeAtMedicalEvent']=HealthCareDataSet['AgeAtMedicalEvent'].dt.total_seconds() / (365.25 * 24 * 60 * 60)

In [10]:
HealthCareDataSet.groupby('County').count()

,Id,MemberName,MemberID,MedicalClaim,ClaimItem,HospitalName,HospitalType,StartDate,EndDate,PrincipalDiagnosisDesc,PrincipalDiagnosis,RevenueCodeDesc,RevenueCode,TypeFlag,BirthDate,TotalExpenses,AgeAtMedicalEvent
County,,,,,,,,,,,,,,,,,
02af982d,23525,23525,23525,23525,23525,23525,23525,23525,23525,23525,23525,23525,23525,23525,23525,23525,23525
217dc01f,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59,59
33b7d74d,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
39825de7,165,165,165,165,165,165,165,165,165,165,165,165,165,165,165,165,165
425a37b2,9468,9468,9468,9468,9468,9468,9468,9468,9468,9468,9468,9468,9468,9468,9468,9468,9468
5597ffc0,443,443,443,443,443,443,443,443,443,443,443,443,443,443,443,443,443
6f0b5b6c,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555,555
6f943458,1849,1849,1849,1849,1849,1849,1849,1849,1849,1849,1849,1849,1849,1849,1849,1849,1849
7a56b047,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7,7


In [92]:
HealthCareDataSet['SecondSemester']=HealthCareDataSet['StartDate']>'2020-06-30'

In [93]:
HealthCareDataSet['SecondSemester']=HealthCareDataSet['SecondSemester'].astype(int)

### 3. Cost Evolution (County Unit of Analysis, First Model)
#### We need to determine the evolution of costs accross time. To do this we fit a model controlling for `County` and whether the medical event took place during the second semester of 2020 (`SecondSemester`).

#### For clarity purposes we consider only the top4 Counties in terms of the number of medical events.

In [94]:
### We select the top4 counties in terms of medical events

Top4Counties=HealthCareDataSet.groupby('County').count().sort_values(by='Id',ascending=False).head(4).index
Top4Counties

Index(['02af982d', '425a37b2', 'fd218584', '89e38653'], dtype='object', name='County')

In [95]:
Top4CountiesFilter=HealthCareDataSet['County'].isin(Top4Counties)

In [96]:
HealthCareDataSet[Top4CountiesFilter].head(10)

,Id,MemberName,MemberID,County,MedicalClaim,ClaimItem,HospitalName,HospitalType,StartDate,EndDate,PrincipalDiagnosisDesc,PrincipalDiagnosis,RevenueCodeDesc,RevenueCode,TypeFlag,BirthDate,TotalExpenses,AgeAtMedicalEvent,FirstSemester,SecondSemester
27,634412,511b5fce,a191c6bb,fd218584,d38d75f3f0c8ff1d,4,446442f4,HOSPITAL,2020-01-24,2020-01-24,Other chronic pain,G89.29,DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DET...,636.0,ER,1969-09-03,9.163,50.390144,1,0
28,634413,511b5fce,a191c6bb,fd218584,d38d75f3f0c8ff1d,5,446442f4,HOSPITAL,2020-01-24,2020-01-24,Other chronic pain,G89.29,DRUGS REQUIRE SPECIFIC ID: DRUGS REQUIRING DET...,636.0,ER,1969-09-03,11.046,50.390144,1,0
29,634414,511b5fce,a191c6bb,fd218584,d38d75f3f0c8ff1d,2,446442f4,HOSPITAL,2020-01-24,2020-01-24,Other chronic pain,G89.29,EMERGENCY ROOM,450.0,ER,1969-09-03,736.750,50.390144,1,0
30,634415,511b5fce,a191c6bb,fd218584,d38d75f3f0c8ff1d,3,446442f4,HOSPITAL,2020-01-24,2020-01-24,Other chronic pain,G89.29,EMERGENCY ROOM,450.0,ER,1969-09-03,2248.225,50.390144,1,0
31,634416,511b5fce,a191c6bb,fd218584,d38d75f3f0c8ff1d,1,446442f4,HOSPITAL,2020-01-24,2020-01-24,Other chronic pain,G89.29,PHARMACY,250.0,ER,1969-09-03,12.922,50.390144,1,0
71,634516,c9e467c7,0abf7923,fd218584,ca0a1759b415e2cc,3,446442f4,HOSPITAL,2020-01-03,2020-01-03,Shortness of breath,R06.02,LABORATORY - CLINICAL DIAGNOSTIC: CHEMISTRY,301.0,ER,1948-01-06,691.775,71.991786,1,0
72,634517,c9e467c7,0abf7923,fd218584,ca0a1759b415e2cc,6,446442f4,HOSPITAL,2020-01-03,2020-01-03,Shortness of breath,R06.02,LABORATORY - CLINICAL DIAGNOSTIC: BACTERIOLOGY...,306.0,ER,1948-01-06,495.950,71.991786,1,0
73,634518,c9e467c7,0abf7923,fd218584,2f6e3fb72ac69afc,11,446442f4,HOSPITAL,2020-01-08,2020-01-10,Other pulmonary embolism,I26.99,DRUGS REQUIRE SPECIFIC ID: SELF ADMIN DRUGS (I...,637.0,INP,1948-01-06,196.308,72.005476,1,0
75,634528,c9e467c7,0abf7923,fd218584,ca0a1759b415e2cc,9,446442f4,HOSPITAL,2020-01-03,2020-01-03,Shortness of breath,R06.02,EMERGENCY ROOM,450.0,ER,1948-01-06,2493.225,71.991786,1,0
76,634529,c9e467c7,0abf7923,fd218584,ca0a1759b415e2cc,7,446442f4,HOSPITAL,2020-01-03,2020-01-03,Shortness of breath,R06.02,RADIOLOGY - DIAGNOSTIC: CHEST X-RAY,324.0,ER,1948-01-06,890.925,71.991786,1,0


### 3.1. Model Fit

In [97]:
HealthCareDataSetGroupedByMedicalClaim=HealthCareDataSet[Top4CountiesFilter].groupby(['County','SecondSemester','MedicalClaim','TypeFlag']).agg({'TotalExpenses':'sum'}).reset_index()
HealthCareDataSetGroupedByMedicalClaim.rename(columns={'TotalExpenses':'TotalExpensesPerClaim'},inplace=True)

In [98]:
HealthCareDataSetGroupedByMedicalClaim

,County,SecondSemester,MedicalClaim,TypeFlag,TotalExpensesPerClaim
0,02af982d,0,0100acd166512fa8,INP,34893.131
1,02af982d,0,014e756981adbe8a,ER,31548.433
2,02af982d,0,01f7100f8a7a575a,ER,30661.225
3,02af982d,0,0217915ce58746a2,INP,23430.722
4,02af982d,0,0239fb736d3c6fc1,INP,20221.579
...,...,...,...,...,...
2704,fd218584,1,f6d813e25b069ea4,ER,3499.167
2705,fd218584,1,f9190d674031fe94,INP,34240.122
2706,fd218584,1,fd02e7498473245d,ER,2145.185
2707,fd218584,1,fee6cebc72f627b0,ER,20669.208


In [99]:
 #We impose a simple, linear, model:
# We specify TotalExpensesPerClaim as the response variable (a.k.a dependent variable).
reg = smf.ols(formula='np.log(TotalExpensesPerClaim) ~ SecondSemester+County+TypeFlag', data=HealthCareDataSetGroupedByMedicalClaim)


In [100]:
results=reg.fit()

In [101]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     np.log(TotalExpensesPerClaim)   R-squared:                       0.536
Model:                                       OLS   Adj. R-squared:                  0.535
Method:                            Least Squares   F-statistic:                     624.3
Date:                           Fri, 09 Aug 2024   Prob (F-statistic):               0.00
Time:                                   09:55:27   Log-Likelihood:                -3706.5
No. Observations:                           2709   AIC:                             7425.
Df Residuals:                               2703   BIC:                             7460.
Df Model:                                      5                                         
Covariance Type:                       nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept              8.9467      0.035    257.862      0.000       8.879       9.015
County[T.425a37b2]    -0.0877      0.047     -1.885      0.060      -0.179       0.004
County[T.89e38653]    -0.0238      0.065     -0.364      0.716      -0.152       0.104
County[T.fd218584]     0.2449      0.058      4.211      0.000       0.131       0.359
TypeFlag[T.INP]        2.0429      0.037     55.362      0.000       1.971       2.115
SecondSemester        -0.0393      0.037     -1.074      0.283      -0.111       0.032
==============================================================================
Omnibus:                       39.907   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               46.059
Skew:                          -0.240   Prob(JB):                     9.96e-11
Kurtosis:                       3.422   Cond. No.                         4.94
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 3.2. Model Interpretation
##### Based on the previous we have fitted the following model:

$ log(TotalExpensesPerClaim)=8.94-0.087*County_{425a37b2}-0.023*County_{89e38653}+0.244*County_{fd218584}+2.042TypeFlag_{INP}-0.039*SecondSemester+u $

#### the p-values for the coefficients of Counties: 425a37b2 and 89e38653 are statistically not significant.

#### The p-value for the coefficient of SecondSemester is not statistically significant.


#### Based on the above we conclude that:

* There is not statistically significant difference in costs between the base County 02af982d and Counties 425a37b2 and 89e38653
* We are finding statistical evidence for a cost difference between the base county 02af982d and fd218584. Costs for County fd218584 are 24% larger than those of 02af982d
* We are finding statistical evidence for a cost difference between the INP and the base category (ER).Costs for InPatient medical event are 200% larger than those of ER services

* There is not statistically significant difference in costs between the first semester and the second semester of 2020.


### 4. Cost Evolution (County, Unit of Analysis, Second Model)

#### We need to determine the evolution of costs accross time. To do this we fit a model controlling for `County` and whether the medical event took place during the second semester of 2020 (`SecondSemester`). This time we include an interaction term to determine if the evolution of costs is contingent on the county under consideration.

#### For clarity purposes we consider only the top4 Counties in terms of the number of medical events.

In [102]:
#We impose a simple, linear, model:
# We specify TotalExpensesPerClaim as the response variable (a.k.a dependent variable). We set AgeAtMedicalEvent as the independent variable.

reg2 = smf.ols(formula='np.log(TotalExpensesPerClaim) ~ SecondSemester+County+TypeFlag+TypeFlag*SecondSemester', data=HealthCareDataSetGroupedByMedicalClaim)


In [103]:
results2=reg2.fit()

In [104]:
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                  OLS Regression Results                                 
=========================================================================================
Dep. Variable:     np.log(TotalExpensesPerClaim)   R-squared:                       0.537
Model:                                       OLS   Adj. R-squared:                  0.536
Method:                            Least Squares   F-statistic:                     522.0
Date:                           Fri, 09 Aug 2024   Prob (F-statistic):               0.00
Time:                                   09:58:21   Log-Likelihood:                -3703.8
No. Observations:                           2709   AIC:                             7422.
Df Residuals:                               2702   BIC:                             7463.
Df Model:                                      6                                         
Covariance Type:                       nonrobust                                         
==================================================================================================
                                     coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------
Intercept                          8.9839      0.038    235.209      0.000       8.909       9.059
County[T.425a37b2]                -0.0886      0.046     -1.906      0.057      -0.180       0.003
County[T.89e38653]                -0.0214      0.065     -0.328      0.743      -0.150       0.107
County[T.fd218584]                 0.2416      0.058      4.157      0.000       0.128       0.356
TypeFlag[T.INP]                    1.9581      0.052     37.705      0.000       1.856       2.060
SecondSemester                    -0.1149      0.049     -2.346      0.019      -0.211      -0.019
TypeFlag[T.INP]:SecondSemester     0.1706      0.074      2.319      0.020       0.026       0.315
==============================================================================
Omnibus:                       38.453   Durbin-Watson:                   2.047
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.640
Skew:                          -0.240   Prob(JB):                     3.34e-10
Kurtosis:                       3.396   Cond. No.                         6.70
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### 4.2. Model Interpretation
##### Based on the previous we have fitted the following model:

$ log(TotalExpensesPerClaim)=8.98-0.088*County_{425a37b2}-0.021*County_{89e38653}+0.24*County_{fd218584}+1.95TypeFlag_{INP}-0.11*SecondSemester+0.17TypeFlag_{INP}*SecondSemester+u $

#### the p-values for the coefficients of Counties: 425a37b2 and 89e38653 are statistically not significant.


#### Based on the above we conclude that:

* There is not statistically significant difference in costs between the base County 02af982d and Counties 425a37b2 and 89e38653
* We are finding statistical evidence for a cost difference between the base county 02af982d and fd218584. Costs for County fd218584 are 24% larger than those of the base category 02af982d


* We are finding statistical evidence for a cost difference between the INP and the base category (ER).Costs for InPatient medical event are 200% larger than those of ER services

* There is not statistically significant difference in costs between the first semester and the second semester of 2020.

* Given the value of the coefficient associated to the interation term `TypeFlag_INP*SecondSemester` (+0.17) we conclude that the increase of costs associated to InPatient medical events have increased 17% during the second semester.